In [17]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Column, DataFrame, SparkSession}
import org.joda.time.format.DateTimeFormat

import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Column, DataFrame, SparkSession}
import org.joda.time.format.DateTimeFormat


In [18]:
 def clearMsisdnLength(msisdn : Column) : Column = {

    when(length(trim(msisdn)) === 9, concat(lit("0"), trim(msisdn)))
      .when(length(trim(msisdn)) === 8, concat(lit("07"), trim(msisdn)))
      .when(length(trim(msisdn)) === 7, concat(lit("070"), trim(msisdn)))
      .otherwise(trim(msisdn))

  }

clearMsisdnLength: (msisdn: org.apache.spark.sql.Column)org.apache.spark.sql.Column


In [27]:
val wom = 1
val taux_agent = 0.04

wom: Int = 1
taux_agent: Double = 0.04


In [22]:
var df_cashin = spark.read.option("delimiter", ";").option("header", "true")
                .csv("/Users/diakite/Downloads/Telegram Desktop/df_cashin_agent.csv")

df_cashin.show()

+----------+----------------------+
|COMPTE_PDV|OBJECTIF_REALISE_AGENT|
+----------+----------------------+
|0787559774|             2106980.0|
|0708645873|             6427800.0|
|0778685928|             2236800.0|
|0787006051|             4038100.0|
|0708108679|              160700.0|
|0798622547|             1564239.0|
|0778620216|              876500.0|
|0749039438|             1067300.0|
|0758911590|             4952350.0|
|0767511919|              727580.0|
|0787001023|             1161300.0|
|0788680404|              892600.0|
|0789909077|           1.8462975E7|
|0789954766|             7301308.0|
|0797757598|             2524550.0|
|0778789394|             1476150.0|
|0779058769|              516250.0|
|0759833189|              483500.0|
|0777235113|              808150.0|
|0787975305|             2741200.0|
+----------+----------------------+
only showing top 20 rows



df_cashin: org.apache.spark.sql.DataFrame = [COMPTE_PDV: string, OBJECTIF_REALISE_AGENT: string]


In [23]:
var df_cashout = spark.read.option("delimiter", ";").option("header", "true")
                .csv("/Users/diakite/Downloads/Telegram Desktop/df_cashout_agent.csv")

df_cashout.show()

+----------+----------------------+
|COMPTE_PDV|OBJECTIF_REALISE_AGENT|
+----------+----------------------+
|0759082838|              330000.0|
|0747937188|             2112100.0|
|0777235113|              748450.0|
|0749039438|              537950.0|
|0749269890|              531500.0|
|0787003769|             2571440.0|
|0758715161|             1229600.0|
|0708195689|             1600250.0|
|0788911201|              114980.0|
|0758899470|             2450700.0|
|0789714907|             4603000.0|
|0767659889|             1386350.0|
|0787100839|             4663200.0|
|0768464849|             1761650.0|
|0788636821|             2370750.0|
|0777522661|              777350.0|
|0768698282|             1005700.0|
|0768661982|             1627735.0|
|0702140389|             3492100.0|
|0787801140|              1.1263E7|
+----------+----------------------+
only showing top 20 rows



df_cashout: org.apache.spark.sql.DataFrame = [COMPTE_PDV: string, OBJECTIF_REALISE_AGENT: string]


In [24]:
var df_canceled = spark.read.option("delimiter", ";").option("header", "true")
                .csv("/Users/diakite/Downloads/Telegram Desktop/df_canceled_agent.csv")

df_canceled.show()

+----------+----------------------+
|COMPTE_PDV|MONTANT_CANCELED_AGENT|
+----------+----------------------+
|0789954766|               65850.0|
|0767172896|               10000.0|
|0787100839|               12200.0|
|0758933672|                5000.0|
|0769101071|                7000.0|
|0759082946|               90000.0|
|0779903830|               20000.0|
|0789714907|               10300.0|
|0788370513|               20000.0|
|0779058769|               10000.0|
|0789909077|                5100.0|
|0778011005|                2000.0|
|0709815075|              100000.0|
|0787512345|               62350.0|
|0797824948|                5000.0|
|0768379585|                3000.0|
|0779192041|               15500.0|
|0787390812|               20300.0|
|0779870781|               10200.0|
|0778638616|               40000.0|
+----------+----------------------+
only showing top 20 rows



df_canceled: org.apache.spark.sql.DataFrame = [COMPTE_PDV: string, MONTANT_CANCELED_AGENT: string]


In [29]:
df_cashin = df_cashin
        .join(df_canceled, Seq("COMPTE_PDV"), "left_outer")
        .withColumn("OBJECTIF_REALISE_AGENT", col("OBJECTIF_REALISE_AGENT").cast("double"))
        .withColumn("MONTANT_CANCELED_AGENT", col("MONTANT_CANCELED_AGENT").cast("double"))
        .na.fill(0, Array("MONTANT_CANCELED_AGENT"))
        .dropDuplicates()
        .withColumn("OBJECTIF_REALISE_AGENT",
          col("OBJECTIF_REALISE_AGENT") - col("MONTANT_CANCELED_AGENT"))
        .select("COMPTE_PDV", "OBJECTIF_REALISE_AGENT")
        .withColumn("OBJECTIF_REALISE_AGENT", col("OBJECTIF_REALISE_AGENT").cast("double"))


df_cashin: org.apache.spark.sql.DataFrame = [COMPTE_PDV: string, OBJECTIF_REALISE_AGENT: double]


In [30]:
df_cashin.show(5)

+----------+----------------------+
|COMPTE_PDV|OBJECTIF_REALISE_AGENT|
+----------+----------------------+
|0708180729|             1470500.0|
|0708848245|             2743150.0|
|0789204103|             1735200.0|
|0747506039|              546600.0|
|0758906083|              955800.0|
+----------+----------------------+
only showing top 5 rows



In [25]:
var df_commission_fixe = spark.read.option("delimiter", ";").option("header", "true")
                .csv("/Users/diakite/Downloads/Telegram Desktop/df_commission_fixe_agent.csv")

df_commission_fixe.show()

+----------+---------------------+
|COMPTE_PDV|COMMISSION_AGENT_FIXE|
+----------+---------------------+
|0702180853|               3471.0|
|0708631904|              21566.0|
|0767172896|               1342.0|
|0767917334|               4075.0|
|0749719236|                242.0|
|0758728060|               9483.0|
|0758991293|              13782.0|
|0759337900|                226.0|
|0767167583|              24790.0|
|0767437833|                209.0|
|0769101071|               3803.0|
|0778164472|              29692.0|
|0778685928|              16876.0|
|0778689196|                210.0|
|0778923758|                338.0|
|0788586525|               4581.0|
|0702140389|              41008.0|
|0707834612|               5904.0|
|0788809659|               4855.0|
|0789452660|              21134.0|
+----------+---------------------+
only showing top 20 rows



df_commission_fixe: org.apache.spark.sql.DataFrame = [COMPTE_PDV: string, COMMISSION_AGENT_FIXE: string]


clearMsisdnLength: (msisdn: org.apache.spark.sql.Column)org.apache.spark.sql.Column


In [ ]:
var df_objectif_cashin = spark.read.option("delimiter", ";").option("header", "true")
                .csv("/Users/diakite/Downloads/Telegram Desktop/objectifPDV_CASHIN_202208.csv")

df_objectif_cashin.show(5)


 for(col <- df_objectif_cashin.columns){
        df_objectif_cashin = df_objectif_cashin.withColumnRenamed(col,col.replaceAll("\\s", ""))
      }

      df_objectif_cashin = df_objectif_cashin
        .withColumn(s"OBJECTIF_S1", regexp_replace(col(s"OBJECTIF_S1"), " ", ""))
        .withColumn(s"OBJECTIF_S1", regexp_replace(col(s"OBJECTIF_S1"), ",", "."))
        .withColumn(s"OBJECTIF_S1", col(s"OBJECTIF_S1").cast("double"))
        .withColumn(s"MSISDN", clearMsisdnLength(col("MSISDN")))
        .withColumn(s"PARENT_USER_MSISDN", clearMsisdnLength(col("PARENT_USER_MSISDN")))
        .withColumn(s"OWNER_MSISDN", clearMsisdnLength(col("OWNER_MSISDN")))
        .withColumn("TYPE_TRANSACTION", lit("CASHIN"))
        .withColumnRenamed(s"OBJECTIF_S1", "OBJECTIF_AGENT")
        .withColumnRenamed("MSISDN", "COMPTE_PDV")

In [46]:
var df_objectif_cashin = spark.read.option("delimiter", ";").option("header", "true")
                .csv("/Users/diakite/Downloads/Telegram Desktop/objectifPDV_CASHIN_202208.csv")

df_objectif_cashin.show(5)

+--------------+---------+------------------+------------+-----------+-----------+-----------+-----------+-----------+
|       USER_ID|   MSISDN|PARENT_USER_MSISDN|OWNER_MSISDN|OBJECTIF_S1|OBJECTIF_S2|OBJECTIF_S3|OBJECTIF_S4|OBJECTIF_S5|
+--------------+---------+------------------+------------+-----------+-----------+-----------+-----------+-----------+
|CIPT0000149392|707041526|         707045293|   707046783|   13174611|   12349561|   10658525|   10060645|    4261372|
|CIPT0000161149|707043165|         707043159|   707046783|    9658280|    9053438|    7813742|    7375438|    3124003|
|CIPT0000149525|707045314|         707046835|   707046783|    7761148|    7275112|    6278924|    5926714|    2510369|
|CIPT0000149474|707046972|         707046824|   707046783|    7117496|    6671768|    5758197|    5435197|    2302178|
|CIPT0000161138|707048520|         707048451|   707046783|    8027166|    7524471|    6494138|    6129856|    2596413|
+--------------+---------+------------------+---

df_objectif_cashin: org.apache.spark.sql.DataFrame = [USER_ID: string, MSISDN: string ... 7 more fields]


In [47]:
 for(col <- df_objectif_cashin.columns){
        df_objectif_cashin = df_objectif_cashin.withColumnRenamed(col,col.replaceAll("\\s", ""))
      }

      df_objectif_cashin = df_objectif_cashin
        .withColumn(s"OBJECTIF_S1", regexp_replace(col(s"OBJECTIF_S1"), " ", ""))
        .withColumn(s"OBJECTIF_S1", regexp_replace(col(s"OBJECTIF_S1"), ",", "."))
        .withColumn(s"OBJECTIF_S1", col(s"OBJECTIF_S1").cast("double"))
        .withColumn(s"MSISDN", clearMsisdnLength(col("MSISDN")))
        .withColumn(s"PARENT_USER_MSISDN", clearMsisdnLength(col("PARENT_USER_MSISDN")))
        .withColumn(s"OWNER_MSISDN", clearMsisdnLength(col("OWNER_MSISDN")))
        .withColumn("TYPE_TRANSACTION", lit("CASHIN"))
        .withColumnRenamed(s"OBJECTIF_S1", "OBJECTIF_AGENT")
        .withColumnRenamed("MSISDN", "COMPTE_PDV")

df_objectif_cashin: org.apache.spark.sql.DataFrame = [USER_ID: string, COMPTE_PDV: string ... 8 more fields]


In [49]:
 var list_pdv_obj_cashin = df_objectif_cashin.select("COMPTE_PDV")
        .orderBy("COMPTE_PDV")
        .distinct()
        .map(r => r.getString(0))
        .collect()
        .toList

println(list_pdv_obj_cashin)

List(0708645873, 0747865384, 0747952596, 0758915327, 0777773859, 0709815075, 0767437833, 0759083654, 0767511919, 0797875735, 0769163078, 0787001023, 0767659889, 0789714907, 0707402203, 0709904540, 0709961628, 0747955583, 0748182374, 0787100839, 0748167741, 0708958892, 0747863445, 0747886357, 0778689196, 0707834612, 0758928829, 0779891982, 0749039438, 0708475957, 0748466414, 0709057400, 0757444446, 0768548916, 0759083542, 0787559774, 0769557618, 0788300091, 0787527561, 0759852507, 0759833189, 0778858392, 0788106428, 0787072267, 0787120637, 0708980694, 0747887701, 0748161046, 0748163928, 0748198404, 0708108679, 0748163889, 0708901837, 0757623837, 0779481952, 0779905688, 0709384633, 0759082838, 0779028734, 0788554564, 0788809659, 0768661982, 0759062703, 0707777488, 0759062687, 0708195689, 0777326963, 0787801140, 0787975305, 0788812437, 0789909077, 0798190677, 0749394575, 0758907442, 0758928633, 0788370293, 0757974133, 0778133276, 0769101071, 0788911201, 0789708694, 0767917334, 0779021423,

 0757882223, 0768006476, 0778469337, 0778699206, 0707917116, 0758903709, 0787406483, 0787942265, 0788051273, 0707346644, 0757685425, 0758933639, 0767029666, 0769487242, 0758150032, 0767325799, 0779670225, 0789488837, 0769202588, 0748684347, 0757730873, 0709665497, 0778166765, 0747952279, 0788899652, 0768456372, 0702180924, 0707015409, 0757629598, 0747118875, 0758907127, 0788888304, 0747068682, 0787123100, 0789156362, 0797286077, 0758690371, 0789048180, 0788228035, 0708977617, 0779335100, 0788887715, 0758989457, 0767237251, 0748344304, 0779030642, 0787521586, 0788693739, 0778700108, 0777536118, 0708454191, 0708851106, 0767781567, 0778225163, 0779860944, 0779923823, 0757105158, 0702150488, 0748748950, 0708849038, 0757288501, 0758014147, 0769302230, 0787385542, 0789165734, 0789406556, 0749624718, 0789164788, 0707712337, 0702134829, 0708518535, 0768386716, 0779512602, 0767488262, 0701683471, 0779854336, 0747692039, 0759185453, 0778275337, 0788600730, 0749706557, 0747205669, 0778856798, 077

9573070, 0758709325, 0787119861, 0707930975, 0789192118, 0798676092, 0787528735, 0708965198, 0759081015, 0759154550, 0749743631, 0789997493, 0708505833, 0787100604, 0789324354, 0767664322, 0788678527, 0788793296, 0768159417, 0757189302, 0767167435, 0767413177, 0778837209, 0768379097, 0788888274, 0747279183, 0757461770, 0757740061, 0779622376, 0787559276, 0767472112, 0768800566, 0708076157, 0789798393, 0707652827, 0758710522, 0779860740, 0748665199, 0778296065, 0788552622, 0788552658, 0759082125, 0768290662, 0767244333, 0789165548, 0702144249, 0709581040, 0747142188, 0767722424, 0779063157, 0787100887, 0747172182, 0748257732, 0787398934, 0779905221, 0748077525, 0767333824, 0779838471, 0777680673, 0748223934, 0709053504, 0787529967, 0748390875, 0778821380, 0789322346, 0787515706, 0787724172, 0709899069, 0767534949, 0779946293, 0797520864, 0707746263, 0768034546, 0767534343, 0768663289, 0789161855, 0758519561, 0768211793, 0788752764, 0789165075, 0769316029, 0787621906, 0787264472, 0787558

769, 0788840382, 0789012576, 0709755815, 0779150738, 0747246217, 0747281157, 0758927615, 0707930903, 0747846844, 0777492506, 0788015883, 0779210301, 0787559193, 0759081090, 0779060343, 0779540352, 0788715227, 0757209695, 0767385539, 0768280280, 0759202384, 0767921709, 0778669501, 0789389334, 0777960114, 0779559744, 0798184716, 0757208438, 0779076162, 0789252250, 0778878217, 0787100962, 0788263445, 0788576362, 0769192481, 0769323134, 0757477154, 0779906344, 0787558680, 0788230896, 0789388853, 0788671671, 0777749180, 0778023674, 0797403512, 0787559776, 0788120156, 0788797425, 0767684916, 0747622423, 0787572975, 0798691272, 0789164962, 0777628947, 0759195246, 0787078638, 0757400152, 0759062771, 0789545716, 0779486945, 0747374320, 0788877570, 0779060269, 0707867445, 0749798934, 0747742523, 0768666161, 0709207105, 0747952592, 0747953316, 0748180250, 0748254698, 0748255870, 0777269084, 0778400968, 0748676144, 0748255598, 0759621889, 0747513840, 0787100637, 0702140798, 0777915502, 0779905655,

 0779117332, 0798175447, 0758720837, 0747953323, 0768376276, 0748588343, 0757477624, 0758722738, 0759240388, 0768340798, 0748963414, 0768376762, 0769618316, 0707813985, 0768066234, 0747233943, 0767028130, 0708962303, 0748177368, 0777582071, 0789451021, 0757582231, 0747999671, 0758708656, 0797934871, 0747999951, 0788888073, 0759129002, 0767892485, 0777634670, 0787513486, 0777739398, 0708548821, 0778593990, 0779150787, 0709621701, 0788320044, 0767206591, 0789491843, 0787973425, 0748160298, 0748161123, 0748197326, 0757065714, 0778543210, 0787655014, 0788759153, 0797741798, 0759077896, 0767167837, 0758711597, 0787580715, 0708269176, 0748226787, 0788758489, 0709278287, 0747721936, 0769469412, 0769730681, 0779554808, 0779677041, 0788144486, 0788216912, 0789102880, 0798175478, 0707589957, 0758723184, 0758901025, 0769381225, 0757376661, 0759683480, 0778783391, 0768331458, 0768968679, 0777511483, 0779958995, 0788767360, 0788813717, 0749837942, 0757618641, 0769608750, 0788277831, 0798177754, 070

...

 0779519160, 0788174966, 0789074430, 0702128094, 0767387336, 0778893502, 0779603140, 0797651857, 0798163418, 0707511195, 0708354651, 0787698830, 0787419923, 0789107815, 0789928718, 0798613147, 0767489927, 0767658120, 0779382733, 0787074915, 0787531985, 0787706812, 0787861092, 0789735838, 0707492424, 0749401630, 0758720828, 0778718015, 0789273825, 0789703974, 0748847877, 0749391832, 0757427601, 0758786532, 0758913150, 0767288038, 0769521271, 0789125855, 0708336255, 0708485906, 0748447843, 0758957517, 0778412169, 0788595523, 0748782767, 0767528545, 0767692148, 0788839628, 0797416735, 0748217247, 0769638952, 0779548267, 0787596210, 0709642471, 0749326707, 0788888427, 0759082249, 0767002430, 0747862105, 0748679408, 0749868871, 0749733309, 0707198563, 0779150773, 0779903801, 0779532989, 0768075267, 0797783414, 0747899140, 0767187884, 0768334952, 0787045457, 0767914204, 0798633134, 0768933811, 0767708433, 0767903972, 0787514662, 0788639709, 0747953013, 0778594403, 0768379479, 0707226555, 076

8266150, 0788653030, 0789706815, 0748236270, 0757754003, 0768705703, 0779902278, 0747790725, 0768697023, 0789538437, 0759672192, 0767419484, 0779565950, 0758873521, 0779379965, 0788963988, 0747866962, 0748186151, 0758844249, 0777300177, 0768661631, 0767245442, 0779151027, 0779906730, 0797001951, 0778654995, 0777041315, 0759062778, 0769947822, 0707294782, 0757628697, 0758697827, 0787579901, 0788731539, 0787376940, 0797274525, 0789774271, 0748179485, 0758215909, 0747862542, 0757356920, 0759002930, 0768375965, 0777836683, 0787520399, 0769240018, 0757925132, 0779237492, 0702130010, 0708955293, 0747998461, 0759545603, 0748194356, 0707009794, 0708962270, 0702145374, 0747955584, 0747955611, 0748182206, 0768374720, 0778818980, 0758432913, 0787100435, 0789165712, 0747270777, 0767546201, 0789355599, 0778414514, 0787100833, 0707675184, 0708471962, 0708966613, 0748550612, 0797876541, 0779904135, 0767390057, 0757491159, 0788667346, 0789508317, 0778520634, 0759746053, 0787516887, 0757039883, 0709904

586, 0768467517, 0789638110, 0757713651, 0787006943, 0768406020, 0787512355, 0759409649, 0789505499, 0749998702, 0767517272, 0779580043, 0758704939, 0747864748, 0747865422, 0709574641, 0747865290, 0748166055, 0768376772, 0778593283, 0769516339, 0787973494, 0747159090, 0768656658, 0768779393, 0787959951, 0708973654, 0708967562, 0758034899, 0708971711, 0759200653, 0747998925, 0747999217, 0778599925, 0787653163, 0777711638, 0748200226, 0708540440, 0708546175, 0769473692, 0769421888, 0787100246, 0709132731, 0758706526, 0767576120, 0757474245, 0777749210, 0788504673, 0788889641, 0788511440, 0797808788, 0769274069, 0789165950, 0787039465, 0708901928, 0759045254, 0768375085, 0708900439, 0748197401, 0768375217, 0748852984, 0748676014, 0707822583, 0708918505, 0788789714, 0768621526, 0798177117, 0798199900, 0749548210, 0787658591, 0777518455, 0769480047, 0758597066, 0759642307, 0777293016, 0778760192, 0788335493, 0700495555, 0748129061, 0758705105, 0787630960, 0797321609, 0709691660, 0748248559,

 0778838618, 0757000754, 0709075229, 0757774336, 0758971010, 0779075776, 0767202350, 0709811559, 0787100105, 0788634020, 0708508695, 0747998375, 0749855701, 0707738765, 0707816498, 0747955553, 0747998713, 0748183089, 0778598384, 0778692413, 0757534018, 0767546673, 0768606673, 0768770303, 0789164941, 0767360888, 0779821859, 0788889198, 0789953050, 0797480854, 0777739611, 0757783930, 0778897826, 0748102333, 0759515644, 0707103403, 0787516885, 0702156070, 0788360349, 0767656950, 0787970934, 0787970873, 0788889635, 0708901538, 0708910671, 0747924621, 0769270937, 0789606658, 0747887514, 0748161167, 0708500971, 0747884860, 0779905635, 0788559210, 0748197158, 0779905554, 0779626772, 0748896018, 0788989783, 0759083631, 0779742383, 0779591583, 0787193215, 0759096496, 0789314392, 0707766369, 0778551633, 0788610379, 0789155355, 0759447435, 0768945927, 0788591417, 0759083586, 0777777107, 0759805129, 0769928412, 0789568761, 0759564614, 0709299661, 0768624574, 0789474688, 0789568469, 0789987222, 077

list_pdv_obj_cashin: List[String] = List(0708645873, 0747865384, 0747952596, 0758915327, 0777773859, 0709815075, 0767437833, 0759083654, 0767511919, 0797875735, 0769163078, 0787001023, 0767659889, 0789714907, 0707402203, 0709904540, 0709961628, 0747955583, 0748182374, 0787100839, 0748167741, 0708958892, 0747863445, 0747886357, 0778689196, 0707834612, 0758928829, 0779891982, 0749039438, 0708475957, 0748466414, 0709057400, 0757444446, 0768548916, 0759083542, 0787559774, 0769557618, 0788300091, 0787527561, 0759852507, 0759833189, 0778858392, 0788106428, 0787072267, 0787120637, 0708980694, 0747887701, 0748161046, 0748163928, 0748198404, 0708108679, 0748163889, 0708901837, 0757623837, 0779481952, 0779905688, 0709384633, 0759082838, 0779028734, 0788554564, 0788809659, 0768661982, 0759062703, ...


In [20]:
var df_objectif_cashout = spark.read.option("delimiter", ";").option("header", "true")
                .csv("/Users/diakite/Downloads/Telegram Desktop/objectifPDV_CASHOUT_202208.csv")

df_objectif_cashout.show(5)

+--------------+---------+------------------+------------+-----------+-----------+-----------+-----------+-----------+
|       USER_ID|   MSISDN|PARENT_USER_MSISDN|OWNER_MSISDN|OBJECTIF_S1|OBJECTIF_S2|OBJECTIF_S3|OBJECTIF_S4|OBJECTIF_S5|
+--------------+---------+------------------+------------+-----------+-----------+-----------+-----------+-----------+
|CIPT0000149392|707041526|         707045293|   707046783|    7547355|    7373578|    6561920|    5952279|    2384255|
|CIPT0000161149|707043165|         707043159|   707046783|    3546850|    3465185|    3083749|    2797251|    1120471|
|CIPT0000149525|707045314|         707046835|   707046783|    7011654|    6850212|    6096164|    5529795|    2215024|
|CIPT0000149474|707046972|         707046824|   707046783|    4196509|    4099885|    3648584|    3309609|    1325702|
|CIPT0000161138|707048520|         707048451|   707046783|   12100893|   11822272|   10520917|    9543462|    3822745|
+--------------+---------+------------------+---

df_objectif_cashout: org.apache.spark.sql.DataFrame = [USER_ID: string, MSISDN: string ... 7 more fields]


In [21]:
  for(col <- df_objectif_cashout.columns){
        df_objectif_cashout = df_objectif_cashout.withColumnRenamed(col,col.replaceAll("\\s", ""))
      }

      df_objectif_cashout = df_objectif_cashout
        .withColumn(s"OBJECTIF_S${wom}", regexp_replace(col(s"OBJECTIF_S${wom}"), " ", ""))
        .withColumn(s"OBJECTIF_S${wom}", regexp_replace(col(s"OBJECTIF_S${wom}"), ",", "."))
        .withColumn(s"MSISDN", clearMsisdnLength(col("MSISDN")))
        .withColumn(s"PARENT_USER_MSISDN", clearMsisdnLength(col("PARENT_USER_MSISDN")))
        .withColumn(s"OWNER_MSISDN", clearMsisdnLength(col("OWNER_MSISDN")))
        .withColumn(s"OBJECTIF_S${wom}" , col(s"OBJECTIF_S${wom}").cast("double"))
        .withColumn("TYPE_TRANSACTION", lit("CASHOUT"))
        .withColumnRenamed(s"OBJECTIF_S${wom}", "OBJECTIF_AGENT")
        .withColumnRenamed("MSISDN", "COMPTE_PDV")

df_objectif_cashout: org.apache.spark.sql.DataFrame = [USER_ID: string, COMPTE_PDV: string ... 8 more fields]


## Calcul des objectifs

In [31]:
  var df_final_cashin = df_commission_fixe
        .join(df_objectif_cashin, Seq("COMPTE_PDV"), "left_outer")
        .dropDuplicates()
        .join(df_cashin , Seq("COMPTE_PDV"), "left_outer")
        .dropDuplicates()
        .withColumn("OBJECTIF_REALISE_AGENT", col("OBJECTIF_REALISE_AGENT").cast("double"))
        .withColumn("OBJECTIF_AGENT", col("OBJECTIF_AGENT").cast("double"))
        .na.fill(0, Array("OBJECTIF_REALISE_AGENT", "OBJECTIF_AGENT"))
        .withColumn("R/O_AGENT",
          col("OBJECTIF_REALISE_AGENT").divide(col(s"OBJECTIF_AGENT")))
        .withColumn("TRANSACTION_TAG", lit("CASHIN"))
        .filter(col("R/O_AGENT") >= 1 )
        .withColumn("COMMISSION_AGENT", round(col("COMMISSION_AGENT_FIXE").multiply(lit(taux_agent))))
        .filter(col("COMMISSION_AGENT") > 0 )
        .filter(col("COMPTE_PDV").isNotNull ) // && col("COMPTE_REVENDEUR").isNotNull)
        .na.fill(0, Array("COMMISSION_AGENT")) // col("COMPTE_REVENDEUR"), col("TETE_DE_PONT")
        .select(col("COMPTE_PDV"), col("COMMISSION_AGENT_FIXE"),
          col(s"OBJECTIF_AGENT").as("OBJECTIF_SEMAINE"), col("OBJECTIF_REALISE_AGENT"), col("R/O_AGENT"),
          col("COMMISSION_AGENT").as("PRIME_HEBDOMADAIRE_AGENT"), col("TRANSACTION_TAG")

          //col("USER_GRADE_NAME"),
          //col("USER_DOMAIN_CODE"),col("USER_GRADE_CODE")
        )
        .dropDuplicates()

df_final_cashin: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COMPTE_PDV: string, COMMISSION_AGENT_FIXE: string ... 5 more fields]


In [32]:
df_final_cashin.show()

+----------+---------------------+----------------+----------------------+------------------+------------------------+---------------+
|COMPTE_PDV|COMMISSION_AGENT_FIXE|OBJECTIF_SEMAINE|OBJECTIF_REALISE_AGENT|         R/O_AGENT|PRIME_HEBDOMADAIRE_AGENT|TRANSACTION_TAG|
+----------+---------------------+----------------+----------------------+------------------+------------------------+---------------+
|0797144663|              12938.0|       1185010.0|             1403750.0|1.1845891595851512|                   518.0|         CASHIN|
|0747881775|                 18.0|          1972.0|               10100.0| 5.121703853955375|                     1.0|         CASHIN|
|0709548191|              23698.0|       1635769.0|             2950300.0|1.8036165253162275|                   948.0|         CASHIN|
|0779976204|              17599.0|       1831855.0|             2825900.0| 1.542643931970598|                   704.0|         CASHIN|
|0788550883|                989.0|         60047.0|    

In [33]:
df_final_cashin.count()

res21: Long = 15508


In [34]:
df_final_cashin.select("COMPTE_PDV").distinct().count()

res22: Long = 15508


In [35]:
df_commission_fixe.count()

res23: Long = 39023


In [36]:
 var df_final_cashout = df_commission_fixe
        .join(df_objectif_cashout, Seq("COMPTE_PDV"), "left_outer")
        .dropDuplicates()
        .join(df_cashout , Seq("COMPTE_PDV"), "left_outer")
        .dropDuplicates()
        .withColumn("OBJECTIF_REALISE_AGENT", col("OBJECTIF_REALISE_AGENT").cast("double"))
        .withColumn("OBJECTIF_AGENT", col("OBJECTIF_AGENT").cast("double"))
        .na.fill(0, Array("OBJECTIF_REALISE_AGENT", "OBJECTIF_AGENT"))
        .withColumn("R/O_AGENT",
          col("OBJECTIF_REALISE_AGENT").divide(col(s"OBJECTIF_AGENT")))
        .withColumn("TRANSACTION_TAG", lit("CASHOUT"))
        .filter(col("R/O_AGENT") >= 1 )
        .withColumn("COMMISSION_AGENT", round(col("COMMISSION_AGENT_FIXE").multiply(lit(taux_agent))))
        .filter(col("COMMISSION_AGENT") > 0 )
        .filter(col("COMPTE_PDV").isNotNull ) // && col("COMPTE_REVENDEUR").isNotNull)
        .na.fill(0, Array("COMMISSION_AGENT")) // col("COMPTE_REVENDEUR"), col("TETE_DE_PONT")
        .select(col("COMPTE_PDV"),
          col("COMMISSION_AGENT_FIXE"),
          col(s"OBJECTIF_AGENT").as("OBJECTIF_SEMAINE"), col("OBJECTIF_REALISE_AGENT"), col("R/O_AGENT"),
          col("COMMISSION_AGENT").as("PRIME_HEBDOMADAIRE_AGENT"), col("TRANSACTION_TAG")
          //, col("USER_GRADE_NAME"),
          //  col("USER_DOMAIN_CODE"), col("USER_GRADE_CODE")
        )
        .dropDuplicates()

df_final_cashout: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COMPTE_PDV: string, COMMISSION_AGENT_FIXE: string ... 5 more fields]


In [37]:
df_final_cashout.count()

res24: Long = 15830


In [38]:
df_final_cashout.select("COMPTE_PDV").distinct().count()

res25: Long = 15830


In [39]:
15830 + 15508

res26: Int = 31338


In [40]:
df_final_cashout.union(df_final_cashin).select("COMPTE_PDV").distinct().count()

res27: Long = 20966


In [41]:
21048 - 20966

res28: Int = 82


In [42]:
df_cashout.union(df_cashin).select("COMPTE_PDV").distinct().count()

res29: Long = 39027


In [ ]:
0707127573, 0707408670, 0708854944

In [45]:
df_cashin.filter(col("COMPTE_PDV") === "0707127573").show()

+----------+----------------------+
|COMPTE_PDV|OBJECTIF_REALISE_AGENT|
+----------+----------------------+
+----------+----------------------+



In [47]:
var df = spark.read.option("header","true").option("delimiter", ";")
.csv("/Users/diakite/Downloads/Telegram Desktop/df_transaction_0700094064.csv")

df.show()

org.apache.spark.SparkException:  Job aborted due to stage failure: Task 0 in stage 172.0 failed 1 times, most recent failure: Lost task 0.0 in stage 172.0 (TID 329) (192.168.43.232 executor driver): org.apache.spark.repl.RemoteClassLoaderError: org.apache.spark.sql.catalyst.expressions.Object

In [ ]:
df.select("header_transferdate").show(false)

In [ ]:
        .withColumn(s"OBJECTIF_S${wom}", regexp_replace(col(s"OBJECTIF_S${wom}"), ",", "."))



df.withColumn("header_transferdate", col("header_transferdate").cast("string"))
.withColumn("header_transferdate2", regexp_replace(col("header_transferdate"), ".000Z", ""))
.withColumn("header_transferdate2", regexp_replace(col("header_transferdate2"), "T", " "))
.select("header_transferdate", "header_transferdate2").show(false)

df.withColumn("header_transferdate2", from_utc_timestamp(col("header_transferdate"), "CST"))
.withColumn("header_transferdate2", from_utc_timestamp(col("header_transferdate"), "CST"))
.select("header_transferdate", "header_transferdate2").show(false)




### Analyse FRA/BI GROSSISTE

In [89]:
var df_tawan = spark.read.option("header","true").option("delimiter", ";")
.csv("/Users/diakite/Downloads/Telegram Desktop/PRIME_HEBDO_GROSSISTE_S1_AOUT22_v20220809_PUSH.csv")

df_tawan.show()

var df_y = spark.read.option("header","true").option("delimiter", ";")
.csv("/Users/diakite/Downloads/random_objectif_hebdomadaire_grossiste_202208_1.csv")
.withColumn("TETE_DE_PONT", clearMsisdnLength(col("TETE_DE_PONT")))


df_y.show()


+------------+--------------+----------+----------+----------+----------+-------------------------------+-----------------+-----------------+--------------+--------------+--------------------+--------------+
|OWNER_MSISDN|PARTENAIRE_NEW|    OBJ_CI|    OBJ_CO| valeur_CI| valeur_CO|COMMISSION_FIXE_GROSSISTE_HEBDO|            RO_CI|            RO_CO|PRIME_HEBDO_CI|PRIME_HEBDO_CO|PRIME_VARIABLE_HEBDO|NUMERO_A_PAYER|
+------------+--------------+----------+----------+----------+----------+-------------------------------+-----------------+-----------------+--------------+--------------+--------------------+--------------+
|  0707068208|   LA COQUILLE|  40672981|  21893326|  51392420|  26062050|                         326210|  1,2635518404712| 1,19041072151394|       8155,25|       8155,25|               16310|    0707068208|
|  0707068362|  DISTRICENTRE| 112509205|  98882459| 132443669| 115532445|                         830870| 1,17718073823382| 1,16838159334205|      20771,75|      20771,

df_tawan: org.apache.spark.sql.DataFrame = [OWNER_MSISDN: string, PARTENAIRE_NEW: string ... 11 more fields]
df_y: org.apache.spark.sql.DataFrame = [COMMISSION_GROSSISTE_FIXE: string, COMMISSION_GROSSISTE_REEL: string ... 8 more fields]


In [90]:
df_y.select("COMMISSION_GROSSISTE_REEL", "OBJECTIF_REALISE_GROSSISTE", "OBJECTIF_SEMAINE", "TETE_DE_PONT", "TRANSACTION_TAG").show()



+-------------------------+--------------------------+----------------+------------+---------------+
|COMMISSION_GROSSISTE_REEL|OBJECTIF_REALISE_GROSSISTE|OBJECTIF_SEMAINE|TETE_DE_PONT|TRANSACTION_TAG|
+-------------------------+--------------------------+----------------+------------+---------------+
|                    187.0|                  933650.0|        863470.0|  0747118393|         CASHIN|
|                   8155.0|                26062050.0|      21893326.0|  0707068208|        CASHOUT|
|                     22.0|                  113500.0|         82154.0|  0707593845|         CASHIN|
|                   3606.0|                17592985.0|      15615473.0|  0747118290|         CASHIN|
|                  20772.0|               132443669.0|     112509205.0|  0707068362|         CASHIN|
|                 355502.0|              5780185076.0|    5775191792.0|  0758023754|        CASHOUT|
|                 246461.0|              7966292916.0|    7759615395.0|  0758023975|       

In [91]:
df_tawan.select("OBJ_CI", "OBJ_CO", "PRIME_HEBDO_CI", "PRIME_HEBDO_CO", "OWNER_MSISDN", "RO_CI", "RO_CO").show()

+----------+----------+--------------+--------------+------------+-----------------+-----------------+
|    OBJ_CI|    OBJ_CO|PRIME_HEBDO_CI|PRIME_HEBDO_CO|OWNER_MSISDN|            RO_CI|            RO_CO|
+----------+----------+--------------+--------------+------------+-----------------+-----------------+
|  40672981|  21893326|       8155,25|       8155,25|  0707068208|  1,2635518404712| 1,19041072151394|
| 112509205|  98882459|      20771,75|      20771,75|  0707068362| 1,17718073823382| 1,16838159334205|
|  30671298|  31141456|       7273,75|       7273,75|  0707103146| 1,08043357017365| 1,08079275419878|
| 111585332|  78538488|     22379,975|     22379,975|  0707107925| 1,02840734479331| 1,02841112754806|
| 175473622| 137385909|         32619|             0|  0707109771| 1,08656989481872|0,963258721096353|
|     82154|    140966|        22,125|        22,125|  0707593845|  1,3815517199406|  1,3758636834414|
| 468053264| 509195320|      96176,15|             0|  0708475526| 1,0404

In [92]:
df_tawan.select("OWNER_MSISDN").distinct().count()

res85: Long = 13


In [93]:
df_y.select("TETE_DE_PONT").distinct().count()

res86: Long = 13


### Analyse CASHIN

In [94]:
var df_y_ci = df_y
.select("COMMISSION_GROSSISTE_REEL", "OBJECTIF_REALISE_GROSSISTE", "OBJECTIF_SEMAINE", 
        "TETE_DE_PONT", "TRANSACTION_TAG", "COMMISSION_GROSSISTE_FIXE")
.filter(col("TRANSACTION_TAG") === "CASHIN")

df_y_ci.show()


+-------------------------+--------------------------+----------------+------------+---------------+-------------------------+
|COMMISSION_GROSSISTE_REEL|OBJECTIF_REALISE_GROSSISTE|OBJECTIF_SEMAINE|TETE_DE_PONT|TRANSACTION_TAG|COMMISSION_GROSSISTE_FIXE|
+-------------------------+--------------------------+----------------+------------+---------------+-------------------------+
|                    187.0|                  933650.0|        863470.0|  0747118393|         CASHIN|                   7473.0|
|                     22.0|                  113500.0|         82154.0|  0707593845|         CASHIN|                    885.0|
|                   3606.0|                17592985.0|      15615473.0|  0747118290|         CASHIN|                 144232.0|
|                  20772.0|               132443669.0|     112509205.0|  0707068362|         CASHIN|                 830870.0|
|                 246461.0|              7966292916.0|    7759615395.0|  0758023975|         CASHIN|           

df_y_ci: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COMMISSION_GROSSISTE_REEL: string, OBJECTIF_REALISE_GROSSISTE: string ... 4 more fields]


In [95]:
var df_tawan_ci = df_tawan
.select("OBJ_CI", "PRIME_HEBDO_CI", "OWNER_MSISDN", "RO_CI", "valeur_CI", "COMMISSION_FIXE_GROSSISTE_HEBDO")

df_tawan_ci.show()

+----------+--------------+------------+-----------------+----------+-------------------------------+
|    OBJ_CI|PRIME_HEBDO_CI|OWNER_MSISDN|            RO_CI| valeur_CI|COMMISSION_FIXE_GROSSISTE_HEBDO|
+----------+--------------+------------+-----------------+----------+-------------------------------+
|  40672981|       8155,25|  0707068208|  1,2635518404712|  51392420|                         326210|
| 112509205|      20771,75|  0707068362| 1,17718073823382| 132443669|                         830870|
|  30671298|       7273,75|  0707103146| 1,08043357017365|  33138300|                         290950|
| 111585332|     22379,975|  0707107925| 1,02840734479331| 114755175|                         895199|
| 175473622|         32619|  0707109771| 1,08656989481872| 190664355|                        1304760|
|     82154|        22,125|  0707593845|  1,3815517199406|    113500|                            885|
| 468053264|      96176,15|  0708475526| 1,04045155852177| 486986748|             

df_tawan_ci: org.apache.spark.sql.DataFrame = [OBJ_CI: string, PRIME_HEBDO_CI: string ... 4 more fields]


In [96]:
List(df_tawan_ci.count(), df_y_ci.count())

res89: List[Long] = List(13, 12)


In [97]:
df_tawan_ci
.join(df_y_ci, col("OWNER_MSISDN") === col("TETE_DE_PONT"))
.show()

+----------+--------------+------------+----------------+----------+-------------------------------+-------------------------+--------------------------+----------------+------------+---------------+-------------------------+
|    OBJ_CI|PRIME_HEBDO_CI|OWNER_MSISDN|           RO_CI| valeur_CI|COMMISSION_FIXE_GROSSISTE_HEBDO|COMMISSION_GROSSISTE_REEL|OBJECTIF_REALISE_GROSSISTE|OBJECTIF_SEMAINE|TETE_DE_PONT|TRANSACTION_TAG|COMMISSION_GROSSISTE_FIXE|
+----------+--------------+------------+----------------+----------+-------------------------------+-------------------------+--------------------------+----------------+------------+---------------+-------------------------+
|    863470|       186,825|  0747118393|1,08127670909238|    933650|                           7473|                    187.0|                  933650.0|        863470.0|  0747118393|         CASHIN|                   7473.0|
|     82154|        22,125|  0707593845| 1,3815517199406|    113500|                            

In [98]:
df_tawan_ci
.join(df_y_ci, col("OWNER_MSISDN") === col("TETE_DE_PONT"))
.select("TETE_DE_PONT", "OBJ_CI", "OBJECTIF_SEMAINE", "valeur_CI", "OBJECTIF_REALISE_GROSSISTE", 
        "COMMISSION_GROSSISTE_REEL", "PRIME_HEBDO_CI")
.show()

+------------+----------+----------------+----------+--------------------------+-------------------------+--------------+
|TETE_DE_PONT|    OBJ_CI|OBJECTIF_SEMAINE| valeur_CI|OBJECTIF_REALISE_GROSSISTE|COMMISSION_GROSSISTE_REEL|PRIME_HEBDO_CI|
+------------+----------+----------------+----------+--------------------------+-------------------------+--------------+
|  0747118393|    863470|        863470.0|    933650|                  933650.0|                    187.0|       186,825|
|  0707593845|     82154|         82154.0|    113500|                  113500.0|                     22.0|        22,125|
|  0747118290|  15615473|      15615473.0|  17592985|                17592985.0|                   3606.0|        3605,8|
|  0707068362| 112509205|     112509205.0| 132443669|               132443669.0|                  20772.0|      20771,75|
|  0758023975|7759615395|    7759615395.0|7966292916|              7966292916.0|                 246461.0|        246461|
|  0708475526| 468053264

In [99]:
// DIFF COMMISSION
df_tawan_ci
.join(df_y_ci, col("OWNER_MSISDN") === col("TETE_DE_PONT"))
.select("TETE_DE_PONT", "COMMISSION_FIXE_GROSSISTE_HEBDO", "COMMISSION_GROSSISTE_FIXE")
.withColumn("DIFF", col("COMMISSION_FIXE_GROSSISTE_HEBDO") - col("COMMISSION_GROSSISTE_FIXE"))
.show()

+------------+-------------------------------+-------------------------+-------+
|TETE_DE_PONT|COMMISSION_FIXE_GROSSISTE_HEBDO|COMMISSION_GROSSISTE_FIXE|   DIFF|
+------------+-------------------------------+-------------------------+-------+
|  0747118393|                           7473|                   7473.0|    0.0|
|  0707593845|                            885|                    885.0|    0.0|
|  0747118290|                         144232|                 144232.0|    0.0|
|  0707068362|                         830870|                 830870.0|    0.0|
|  0758023975|                        9858440|                9858440.0|    0.0|
|  0708475526|                        3847046|                3851630.0|-4584.0|
|  0707068208|                         326210|                 326210.0|    0.0|
|  0707103146|                         290950|                 290950.0|    0.0|
|  0747118292|                          65753|                  65753.0|    0.0|
|  0747118464|              

In [100]:
// DIFF OBJECTIF
df_tawan_ci
.join(df_y_ci, col("OWNER_MSISDN") === col("TETE_DE_PONT"))
.select("TETE_DE_PONT", "OBJ_CI", "OBJECTIF_SEMAINE")
.withColumn("DIFF", col("OBJ_CI") - col("OBJECTIF_SEMAINE"))
.show()

// 16915348.0 // 0747118290

+------------+----------+----------------+----+
|TETE_DE_PONT|    OBJ_CI|OBJECTIF_SEMAINE|DIFF|
+------------+----------+----------------+----+
|  0747118393|    863470|        863470.0| 0.0|
|  0707593845|     82154|         82154.0| 0.0|
|  0747118290|  15615473|      15615473.0| 0.0|
|  0707068362| 112509205|     112509205.0| 0.0|
|  0758023975|7759615395|    7759615395.0| 0.0|
|  0708475526| 468053264|     468053264.0| 0.0|
|  0707068208|  40672981|      40672981.0| 0.0|
|  0707103146|  30671298|      30671298.0| 0.0|
|  0747118292|   7076704|       7076704.0| 0.0|
|  0747118464|  17035699|      17035699.0| 0.0|
|  0707107925| 111585332|     111585332.0| 0.0|
|  0707109771| 175473622|     175473622.0| 0.0|
+------------+----------+----------------+----+



In [101]:
// DIFF REALISE
df_tawan_ci
.join(df_y_ci, col("OWNER_MSISDN") === col("TETE_DE_PONT"))
.select("TETE_DE_PONT", "valeur_CI", "OBJECTIF_REALISE_GROSSISTE")
.withColumn("DIFF", col("valeur_CI") - col("OBJECTIF_REALISE_GROSSISTE"))
.show()

+------------+----------+--------------------------+----+
|TETE_DE_PONT| valeur_CI|OBJECTIF_REALISE_GROSSISTE|DIFF|
+------------+----------+--------------------------+----+
|  0747118393|    933650|                  933650.0| 0.0|
|  0707593845|    113500|                  113500.0| 0.0|
|  0747118290|  17592985|                17592985.0| 0.0|
|  0707068362| 132443669|               132443669.0| 0.0|
|  0758023975|7966292916|              7966292916.0| 0.0|
|  0708475526| 486986748|               486986748.0| 0.0|
|  0707068208|  51392420|                51392420.0| 0.0|
|  0707103146|  33138300|                33138300.0| 0.0|
|  0747118292|   7305895|                 7305895.0| 0.0|
|  0747118464|  18171845|                18171845.0| 0.0|
|  0707107925| 114755175|               114755175.0| 0.0|
|  0707109771| 190664355|               190664355.0| 0.0|
+------------+----------+--------------------------+----+



### Analyse CASHOUT

In [102]:
var df_tawan_co = df_tawan
.select("OBJ_CO", "PRIME_HEBDO_CO", "OWNER_MSISDN", "RO_CO", "valeur_CO", "COMMISSION_FIXE_GROSSISTE_HEBDO")

df_tawan_co.show(5)

+---------+--------------+------------+-----------------+---------+-------------------------------+
|   OBJ_CO|PRIME_HEBDO_CO|OWNER_MSISDN|            RO_CO|valeur_CO|COMMISSION_FIXE_GROSSISTE_HEBDO|
+---------+--------------+------------+-----------------+---------+-------------------------------+
| 21893326|       8155,25|  0707068208| 1,19041072151394| 26062050|                         326210|
| 98882459|      20771,75|  0707068362| 1,16838159334205|115532445|                         830870|
| 31141456|       7273,75|  0707103146| 1,08079275419878| 33657460|                         290950|
| 78538488|     22379,975|  0707107925| 1,02841112754806| 80769855|                         895199|
|137385909|             0|  0707109771|0,963258721096353|132338175|                        1304760|
+---------+--------------+------------+-----------------+---------+-------------------------------+
only showing top 5 rows



df_tawan_co: org.apache.spark.sql.DataFrame = [OBJ_CO: string, PRIME_HEBDO_CO: string ... 4 more fields]


In [103]:
var df_y_co = df_y
.select("COMMISSION_GROSSISTE_REEL", "OBJECTIF_REALISE_GROSSISTE", "OBJECTIF_SEMAINE", 
        "TETE_DE_PONT", "TRANSACTION_TAG", "COMMISSION_GROSSISTE_FIXE")
.filter(col("TRANSACTION_TAG") === "CASHOUT")

df_y_co.show()

+-------------------------+--------------------------+----------------+------------+---------------+-------------------------+
|COMMISSION_GROSSISTE_REEL|OBJECTIF_REALISE_GROSSISTE|OBJECTIF_SEMAINE|TETE_DE_PONT|TRANSACTION_TAG|COMMISSION_GROSSISTE_FIXE|
+-------------------------+--------------------------+----------------+------------+---------------+-------------------------+
|                   8155.0|                26062050.0|      21893326.0|  0707068208|        CASHOUT|                 326210.0|
|                 355502.0|              5780185076.0|    5775191792.0|  0758023754|        CASHOUT|               14220076.0|
|                   7274.0|                33657460.0|      31141456.0|  0707103146|        CASHOUT|                 290950.0|
|                   1644.0|                 9041600.0|       8269113.0|  0747118292|        CASHOUT|                  65753.0|
|                     22.0|                  193950.0|        140966.0|  0707593845|        CASHOUT|           

df_y_co: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [COMMISSION_GROSSISTE_REEL: string, OBJECTIF_REALISE_GROSSISTE: string ... 4 more fields]


In [104]:
// DIFF COMMISSION
df_tawan_co
.join(df_y_co, col("OWNER_MSISDN") === col("TETE_DE_PONT"))
.select("TETE_DE_PONT", "COMMISSION_FIXE_GROSSISTE_HEBDO", "COMMISSION_GROSSISTE_FIXE")
.withColumn("DIFF", col("COMMISSION_FIXE_GROSSISTE_HEBDO") - col("COMMISSION_GROSSISTE_FIXE"))
.show()

+------------+-------------------------------+-------------------------+-------+
|TETE_DE_PONT|COMMISSION_FIXE_GROSSISTE_HEBDO|COMMISSION_GROSSISTE_FIXE|   DIFF|
+------------+-------------------------------+-------------------------+-------+
|  0707068208|                         326210|                 326210.0|    0.0|
|  0758023754|                       14220076|               14220076.0|    0.0|
|  0707103146|                         290950|                 290950.0|    0.0|
|  0747118292|                          65753|                  65753.0|    0.0|
|  0707593845|                            885|                    885.0|    0.0|
|  0707107925|                         895199|                 904260.0|-9061.0|
|  0758023975|                        9858440|                9858440.0|    0.0|
|  0707068362|                         830870|                 830870.0|    0.0|
|  0747118290|                         144232|                 144232.0|    0.0|
+------------+--------------

In [105]:
// DIFF OBJECTIF
df_tawan_co
.join(df_y_co, col("OWNER_MSISDN") === col("TETE_DE_PONT"))
.select("TETE_DE_PONT", "OBJ_CO", "OBJECTIF_SEMAINE")
.withColumn("DIFF", col("OBJ_CO") - col("OBJECTIF_SEMAINE"))
.show()

+------------+----------+----------------+----+
|TETE_DE_PONT|    OBJ_CO|OBJECTIF_SEMAINE|DIFF|
+------------+----------+----------------+----+
|  0707068208|  21893326|      21893326.0| 0.0|
|  0758023754|5775191792|    5775191792.0| 0.0|
|  0707103146|  31141456|      31141456.0| 0.0|
|  0747118292|   8269113|       8269113.0| 0.0|
|  0707593845|    140966|        140966.0| 0.0|
|  0707107925|  78538488|      78538488.0| 0.0|
|  0758023975|4584820025|    4584820025.0| 0.0|
|  0707068362|  98882459|      98882459.0| 0.0|
|  0747118290|  16549834|      16549834.0| 0.0|
+------------+----------+----------------+----+



In [106]:
// DIFF REALISE
df_tawan_co
.join(df_y_co, col("OWNER_MSISDN") === col("TETE_DE_PONT"))
.select("TETE_DE_PONT", "valeur_CO", "OBJECTIF_REALISE_GROSSISTE")
.withColumn("DIFF", col("valeur_CO") - col("OBJECTIF_REALISE_GROSSISTE"))
.show()

+------------+----------+--------------------------+------+
|TETE_DE_PONT| valeur_CO|OBJECTIF_REALISE_GROSSISTE|  DIFF|
+------------+----------+--------------------------+------+
|  0707068208|  26062050|                26062050.0|   0.0|
|  0758023754|5780190076|              5780185076.0|5000.0|
|  0707103146|  33657460|                33657460.0|   0.0|
|  0747118292|   9041600|                 9041600.0|   0.0|
|  0707593845|    193950|                  193950.0|   0.0|
|  0707107925|  80769855|                80769855.0|   0.0|
|  0758023975|4598208663|              4598208663.0|   0.0|
|  0707068362| 115532445|               115532445.0|   0.0|
|  0747118290|  17515030|                17515030.0|   0.0|
+------------+----------+--------------------------+------+



In [ ]:
var list_pdv_obj_cashin = df_objectif_cashin.select("MSISDN")
        .orderBy("MSISDN")
        .distinct()
        .map(r => r.getString(0))
        .collect()
        .toList

println(list_pdv_obj_cashin)

### Checking OBJECTIF 

In [32]:
val wom = 1

wom: Int = 1


In [39]:
var df_cash = spark.read.option("delimiter", ";").option("header", "true")
                .csv("/Users/diakite/Downloads/Telegram Desktop/objectifPDV_CASHIN_202208.csv")

// df_cash.show(5)

 for(col <- df_cash.columns){
        df_cash = df_cash.withColumnRenamed(col,col.replaceAll("\\s", ""))
      }

      df_cash = df_cash
        .withColumn(s"OBJECTIF_S1", regexp_replace(col(s"OBJECTIF_S1"), " ", ""))
        .withColumn(s"OBJECTIF_S1", regexp_replace(col(s"OBJECTIF_S1"), ",", "."))
        .withColumn(s"OBJECTIF_S1", col(s"OBJECTIF_S1").cast("double"))
        .withColumn(s"MSISDN", clearMsisdnLength(col("MSISDN")))
        .withColumn(s"PARENT_USER_MSISDN", clearMsisdnLength(col("PARENT_USER_MSISDN")))
        .withColumn(s"OWNER_MSISDN", clearMsisdnLength(col("OWNER_MSISDN")))
        .withColumn("TYPE_TRANSACTION", lit("CASHIN"))
        .withColumnRenamed("MSISDN", "COMPTE_PDV")
        .groupBy(col("OWNER_MSISDN").as("TETE_DE_PONT"))
          .agg(sum(col(s"OBJECTIF_S1")).as("OBJECTIF_GROSSISTE"))

df_cash.filter(col("TETE_DE_PONT") === "0747118290").show(5)

+------------+------------------+
|TETE_DE_PONT|OBJECTIF_GROSSISTE|
+------------+------------------+
|  0747118290|       1.6915348E7|
+------------+------------------+



df_cash: org.apache.spark.sql.DataFrame = [TETE_DE_PONT: string, OBJECTIF_GROSSISTE: double]
df_cash: org.apache.spark.sql.DataFrame = [TETE_DE_PONT: string, OBJECTIF_GROSSISTE: double]


In [3]:
var tmp = spark.read
                .option("header", "true")
                .option("delimiter", ";")
                .csv("/Users/diakite/Downloads/Telegram Desktop/TRANSACTIONS_MOOD_ABSENTES_GR_HOURLY_20220906_22H.csv")
            
tmp.show(5)

+------+---------+------------+---+----------+----+------------+------+-----+----------+------------+------------------+-------+--------------------+-------------------+----+----+-----+
|amount|categorie|categorycode|day|grade_name|hour|loading_date|minute|month|    msisdn|owner_msisdn|parent_user_msisdn|    tag|          transferid|         transferon|week|year|hour2|
+------+---------+------------+---+----------+----+------------+------+-----+----------+------------+------------------+-------+--------------------+-------------------+----+----+-----+
|  5100|     null|     PDVTYP2|  6|      null|   5|  2022-09-06|    40|    9|0788393916|  0787648106|        0787648132|CASHOUT|CO220905.2240.B00900|2022-09-05 22:40:58|   1|2022|   22|
|145907|     null|     PDVTYP2|  6|      null|   5|  2022-09-06|    40|    9|0767600920|  0758025342|        0757119303| CASHIN|CI220905.2243.C66328|2022-09-05 22:43:35|   1|2022|   22|
| 45400|     null|     PDVTYP2|  6|      null|   5|  2022-09-06|    40

tmp: org.apache.spark.sql.DataFrame = [amount: string, categorie: string ... 16 more fields]


In [8]:
tmp.select("transferon").withColumn("tmp", substring(col("transferon"), 12, 2))
.withColumn("tmp2", substring(col("transferon"), 15, 2))
.withColumn("tmp3", substring(col("transferon"), 1, 10))

.show()

+-------------------+---+----+----------+
|         transferon|tmp|tmp2|      tmp3|
+-------------------+---+----+----------+
|2022-09-05 22:40:58| 22|  40|2022-09-05|
|2022-09-05 22:43:35| 22|  43|2022-09-05|
|2022-09-05 22:06:25| 22|  06|2022-09-05|
|2022-09-05 22:56:46| 22|  56|2022-09-05|
|2022-09-05 22:11:57| 22|  11|2022-09-05|
|2022-09-05 22:32:54| 22|  32|2022-09-05|
|2022-09-05 22:09:35| 22|  09|2022-09-05|
|2022-09-05 22:12:37| 22|  12|2022-09-05|
|2022-09-05 22:24:32| 22|  24|2022-09-05|
|2022-09-05 22:22:00| 22|  22|2022-09-05|
|2022-09-05 22:22:35| 22|  22|2022-09-05|
|2022-09-05 22:21:19| 22|  21|2022-09-05|
|2022-09-05 22:43:11| 22|  43|2022-09-05|
|2022-09-05 22:02:43| 22|  02|2022-09-05|
|2022-09-05 22:36:27| 22|  36|2022-09-05|
|2022-09-05 22:13:40| 22|  13|2022-09-05|
|2022-09-05 22:00:53| 22|  00|2022-09-05|
|2022-09-05 22:18:52| 22|  18|2022-09-05|
|2022-09-05 22:59:46| 22|  59|2022-09-05|
|2022-09-05 22:19:49| 22|  19|2022-09-05|
+-------------------+---+----+----

In [ ]:
var df_cashin = spark.read.option("delimiter", ";").option("header", "true")
                .csv("/Users/diakite/Downloads/Telegram Desktop/df_cashin_agent.csv")

df_cashin.show()